# Simulación de Modelos de Neuronas

Este proyecto se centra en la simulación de varios modelos de neuronas, utilizando algoritmos de aprendizaje automático y técnicas de simulación numérica. Los modelos incluyen:

- Integrate-and-Fire (IF)
- FitzHugh-Nagumo (FHN)
- Integrate-and-Fire con Adaptación (IFA)
- Hodgkin-Huxley (HH)
- Nonlinear Integrate-and-Fire (NLIF)

## Bibliotecas Importadas

En este proyecto se utilizan las siguientes bibliotecas de Python:

1. **NumPy**:
    - `import numpy as np`

2. **Scikit-learn**:
    - `from sklearn.model_selection import train_test_split`
    - `from sklearn.metrics import accuracy_score`
    - Aprendizaje automático en Python. Se utiliza para dividir los datos en conjuntos de entrenamiento y prueba (`train_test_split`) y para calcular la precisión del modelo (`accuracy_score`).

3. **Time**:
    - `import time`
    - Medición y manipulación del tiempo. En este proyecto, se utiliza para medir el tiempo de ejecución de los modelos.


In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import time

# Modelos Neuronales


## Integrate-and-Fire (IF)
La ecuación diferencial que describe la dinámica del potencial de membrana $v$ es:

$$
\frac{dv}{dt} = \frac{-v + i}{\tau}
$$

donde:
- $\tau $ es la constante de tiempo de la membrana.
- $v$ es el potencial de membrana.
- $i$ es la corriente de entrada.

Cuando $v$ alcanza el umbral $v_{th}$, se genera un spike y $v$ se reinicia a $v_{reset}$.

$$
\text{si } v(t) \geq v_{th}, \text{ entonces } v(t) \leftarrow 0
$$


In [2]:
class IF:
    def __init__(self, tau, v_reset, v_th, v_init):
        self.tau = tau
        self.v_reset = v_reset
        self.v_th = v_th
        self.v = v_init

    def update(self, I, dt):
        dvdt = (-self.v + I) / self.tau
        self.v += dvdt * dt
        if self.v >= self.v_th:
            spike = True
            self.v = self.v_reset
        else:
            spike = False
        return spike

## Modelo FitzHugh-Nagumo

Las ecuaciones diferenciales que describen la dinámica del potencial de membrana $V$ y la variable de recuperación $W$ es:

$$
\frac{dv}{dt} = v - \frac{v^3}{3} - w + i
$$

$$
\frac{dw}{dt} = \frac{v + a - bW}{\tau}
$$


donde:
- $v$ es el potencial de membrana.
- $w$ es la variable de recuperación.
- $i$ es la corriente de entrada.
- $a$ es un parámetro que influye en la tasa de recuperación de $w$.
- $b$ es un parámetro que modula la amplitud de $w$.
- $\tau$ es la constante de tiempo asociada con la variable $w$, que determina la rapidez con la que $w$ responde a cambios en $v$.

In [3]:
class FitzHughNagumo:
    def __init__(self, v_init, w_init, a, b, tau):
        self.v = v_init
        self.w = w_init
        self.a = a
        self.b = b
        self.tau = tau

    def update(self, I, dt):
        dvdt = self.v - (self.v ** 3) / 3 - self.w + I
        dwdt = (self.v + self.a - self.b * self.w) / self.tau

        self.v += dvdt * dt
        self.w += dwdt * dt

        spike = self.v >= 1.0
        if spike:
            self.v = 0.0
            self.w += 0.5
        return spike

## Modelo Integrate and Fire con Adaptación

$$
\frac{dv}{dt} = \frac{-v + i - w}{\tau}
$$
$$
\frac{dw}{dt} = \frac{a \cdot v - w}{\tau_w}
$$

donde:
- $\tau$ es la constante de tiempo para la variable $v$.
- $\tau_w$ es la constante de tiempo para la variable de adaptación $w$.
- $v$ es el potencial de membrana.
- $w$ es la variable de adaptación.
- $i$ es la corriente de entrada.
- $a$ es la constante de adaptación que controla la influencia de $w$ sobre $v$.

In [4]:
class IFAdaptacion:
    def __init__(self, tau, tau_w, v_reset, v_th, v_init, w_init, a):
        self.tau = tau
        self.tau_w = tau_w
        self.v_reset = v_reset
        self.v_th = v_th
        self.v = v_init
        self.w = w_init
        self.a = a

    def update(self, I, dt):
        dvdt = (-self.v + I - self.w) / self.tau
        dwdt = (self.a * self.v - self.w) / self.tau_w

        self.v += dvdt * dt
        self.w += dwdt * dt

        spike = self.v >= self.v_th
        if spike:
            self.v = self.v_reset
            self.w += 1.0
        return spike

## Modelo Hodgkin-Huxley

Las ecuaciones diferenciales que describen la dinámica del potencial de membrana $v$, y las variables de compuerta $n$, $m$, y $h$ son:

$$
\frac{dv}{dt} = \frac{I - \bar{g}_{Na} m^3 h (v - E_{Na}) - \bar{g}_{K} n^4 (v - E_{K}) - \bar{g}_{L} (v - E_{L})}{C}
$$

$$
\begin{array}{ccc}
\frac{dn}{dt} = \alpha_n (1 - n) - \beta_n n & \quad & \frac{dm}{dt} = \alpha_m (1 - m) - \beta_m m & \quad & \frac{dh}{dt} = \alpha_h (1 - h) - \beta_h h
\end{array}
$$

donde las constantes $\alpha$ y $\beta$ son funciones del potencial de membrana $v$:

$$
\begin{array}{ccc}
\alpha_n = \frac{0.01 (v + 60)}{1 - e^{-0.1 (v + 60)}} & \beta_n = 0.125 e^{-\frac{(v + 70)}{80}}
\end{array}
$$

$$
\begin{array}{ccc}
\alpha_m = \frac{0.1 (v + 45)}{1 - e^{-0.1 (v + 45)}} & \beta_m = 4 e^{-0.0556 (v + 70)}
\end{array}
$$

$$
\begin{array}{ccc}
\alpha_h = 0.07 e^{-0.05 (v + 70)} & \beta_h = \frac{1}{1 + e^{-0.1 (v + 40)}}
\end{array}
$$

### Parámetros

- $v$ es el potencial de membrana.
- $n$ es la probabilidad de apertura de los canales de potasio.
- $m$ es la probabilidad de apertura de los canales de sodio (activación).
- $h$ es la probabilidad de cierre de los canales de sodio (inactivación).
- $i$ es la corriente externa aplicada.
- $C$ es la capacitancia de la membrana ($\mu F$/$cm^2$).
- $\bar{g}_{Na}$ es la conductancia máxima de los canales de sodio (mS/$cm^2$).
- $\bar{g}_{K}$ es la conductancia máxima de los canales de potasio (mS/$cm^2$).
- $\bar{g}_{L}$ es la conductancia de fuga (mS/$cm^2$).
- $E_{Na}$ es el potencial de equilibrio del sodio ($mV$).
- $E_{K}$ es el potencial de equilibrio del potasio ($mV$).
- $E_{L}$ es el potencial de equilibrio de la fuga ($mV$).

Las conductancias y potenciales de equilibrio específicos son:
- $\bar{g}_{Na} = 120.0$ $mS$/$cm^2$
- $\bar{g}_{K} = 36.0$ $mS$/$cm^2$
- $\bar{g}_{L} = 0.3$ $mS$/$cm^2$
- $E_{Na} = 55.0$ $mV$
- $E_{K} = -82.0$ $mV$
- $E_{L} = -59.0$ $mV$


In [5]:
class HH:
    def __init__(self, v_init, n_init, m_init, h_init):
        self.v = v_init
        self.n = n_init
        self.m = m_init
        self.h = h_init
        self.C = 1.0
        self.g_Na = 120.0
        self.g_K = 36.0
        self.g_L = 0.3
        self.E_Na = 55.0
        self.E_K = -82.0
        self.E_L = -59.0

    def update(self, I, dt):
        alpha_n = (0.01 * (self.v + 60.0)) / (1.0 - np.exp(-0.1 * (self.v + 60.0)))
        beta_n = 0.125 * np.exp(-(1/80) * (self.v + 70.0))
        alpha_m = (0.1 * (self.v + 45.0)) / (1.0 - np.exp(-0.1 * (self.v + 45.0)))
        beta_m = 4.0 * np.exp(-0.0556 * (self.v + 70.0))
        alpha_h = 0.07 * np.exp(-0.05 * (self.v + 70.0))
        beta_h = 1.0 / (1.0 + np.exp(-0.1 * (self.v + 40.0)))

        dvdt = (I - self.g_Na * self.m**3 * self.h * (self.v - self.E_Na) - self.g_K * self.n**4 * (self.v - self.E_K) - self.g_L * (self.v - self.E_L)) / self.C
        dndt = alpha_n * (1.0 - self.n) - beta_n * self.n
        dmdt = alpha_m * (1.0 - self.m) - beta_m * self.m
        dhdt = alpha_h * (1.0 - self.h) - beta_h * self.h

        self.v += dvdt * dt
        self.n += dndt * dt
        self.m += dmdt * dt
        self.h += dhdt * dt

        spike = self.v >= 0.0
        if spike:
            self.v = 30.0
            self.n = 0.3177
            self.m = 0.0529
            self.h = 0.5961
        return spike

## Modelo Integrate and Fire NLIF


\begin{array}{ll}
\tau \frac{\mathrm{d}v(t)}{\mathrm{d}t} = - v(t) + i(t) \\
v(t) \leftarrow
\begin{cases}
v_{reset} + m \cdot (v(t) - v_{th}) & \text{si } v(t) \geq v_{th} \\
v(t) \cdot n & \text{en caso contrario}
\end{cases}
\end{array}



donde:
- $\tau$ es la constante de tiempo para la variable $v$.
- $v$ es el potencial de membrana.
- $m$ es un factor de reseteo si se produce el impulso.
- $n$ es un factor de reajuste si no se produce el impulso; al no alcanzar el umbral, corrige y crea un efecto de caida.

In [6]:
class NLIF:
    def __init__(self, tau, v_reset, v_th, v_init, m, n):
        self.tau = tau
        self.v_reset = v_reset
        self.v_th = v_th
        self.v = v_init
        self.m = m
        self.n = n

    def update(self, I, dt):
        dvdt = (-self.v + I) / self.tau
        self.v += dvdt * dt
        spike = self.v >= self.v_th
        self.v = np.where(spike, self.v_reset + self.m * (self.v - self.v_th), self.v * self.n)
        return spike

# Entrenamiento de modelos y parámetros utilizados

Se invita al observador a interactuar con los parámetros, para investigar posibles optimizaciones de rendimiento.

## Integrate-and-Fire (IF)

### Parámetros:

- tau = 10 milisegundos $(ms)$
- v_reset = 0 milivoltios $(mV)$
- v_th = 0.5


In [7]:
class IFModel:
    def __init__(self):
        self.neuron = IF(tau=10, v_reset=0, v_th=0.5, v_init=0)

    def fit(self, X, y):
        pass

    def predict(self, X):
        predictions = []
        for sample in X:
            spikes = [self.neuron.update(I=ft, dt=1) for ft in sample]
            predictions.append(int(any(spikes)))
        return np.array(predictions)

## FitzHugh-Nagumo (FHN)

### Parámetros:

- v_init =  - 1.0 $mV$
- w_init =  - 0.5 $mV$
- a = 0.7
- b = 0.8
- tau = 12.5 $mS$

In [8]:
class FHNModel:
    def __init__(self):
        self.neuron = FitzHughNagumo(v_init = -1, w_init = -0.5, a = 0.7, b = 0.8, tau = 12.5)
    def fit(self, X, y):
        pass

    def predict(self, X):
        predictions = []
        for sample in X:
            spikes = [self.neuron.update(I=ft, dt=1) for ft in sample]
            predictions.append(int(any(spikes)))
        return np.array(predictions)

## Hodgkin -Huxley (HH)

Los parámetros propios del modelo están descritos arriba en el modelo, al ser puramente biológico.

El resto de parámetros son los parámetros iniciales del voltaje inicial y de los portales de activación:

- v_init = -65 mV
- n_init = -0.317
- m_init = 0.053
- h_init = 0.5961

In [9]:
class HHModel:
    def __init__(self):
        self.neuron = HH(v_init=-65.0, n_init=0.3177, m_init=0.0529, h_init=0.5961)

    def fit(self, X, y):
        pass

    def predict(self, X):
        predictions = []
        for sample in X:
            spikes = [self.neuron.update(I=ft, dt=1) for ft in sample]
            predictions.append(int(any(spikes)))
        return np.array(predictions)

## NLIF

### Parámetros:

- tau = 10 milisegundos $(ms)$
- v_reset = 0 milivoltios $(mV)$
- v_th = 0.5 milivoltios $(mV)$
- v_init=0.0 milivoltios $(mV)$  
- m = 1.0
- n = 1.0

In [10]:
class NLIFModel:
    def __init__(self):
        self.neuron = NLIF(tau=10, v_reset=0, v_th=0.5, v_init=0.0, m=1, n=1)

    def fit(self, X, y):
        pass

    def predict(self, X):
        predictions = []
        for sample in X:
            spikes = [self.neuron.update(I=ft, dt=1) for ft in sample]
            predictions.append(int(any(spikes)))
        return np.array(predictions)

## Integrate and Fire con Adaptación

### Parámetros:

- tau = 10 milisegundos $(ms)$
- tau_w= 200 milisegundos $(ms)$
- v_reset = 0 milivoltios $(mV)$
- v_th = 1
- v_init=0.0 milivoltios $(mV)$
- w_init=0.0 milivoltios $(mV)$   
- a = 1.0

In [11]:
class IFAModel:
    def __init__(self):
        self.neuron = IFAdaptacion(tau= 10, tau_w= 200, v_reset= 0, v_th= 1.0, v_init= 0, w_init=0, a=1)

    def fit(self, X, y):
        pass

    def predict(self, X):
        predictions = []
        for sample in X:
            spikes = [self.neuron.update(I=ft, dt=1) for ft in sample]
            predictions.append(int(any(spikes)))
        return np.array(predictions)

# SIMULACION

# Función Generate_Data:

Esta función genera un conjunto de datos sintéticos con un número `num_samples` de muestras.

La mitad de las muestras se extraen de una distribución normal con media 0 y desviación estándar 1.
La otra mitad se extrae de una distribución normal con media 2 y desviación estándar 7.

Las muestras se concatenan y se mezclan, y se crean las etiquetas correspondientes.

Se invita al usuario a modificar todos los parámetros (media, desviación, número de muestras...etc).

In [12]:
def generate_data(num_samples):
    X1 = np.random.normal(0, 1, size=(num_samples // 2, 8))
    X2 = np.random.normal(2, 7, size=(num_samples // 2, 8))
    X = np.concatenate([X1, X2], axis=0)
    y = np.concatenate([np.zeros(num_samples // 2), np.ones(num_samples // 2)]).astype(int)
    indices = np.random.permutation(num_samples)
    X = X[indices]
    y = y[indices]
    return X, y

# Función evaluate_model:

Sirve para evaluar el rendimiento de cada uno de los modelos previamente descrito.

La función `evaluate_model` toma cada modelo y datos de entrenamiento/prueba como entrada.
 Ajusta el modelo a los datos de entrenamiento, predice las etiquetas para los datos de prueba, y calcula la precisión, el tiempo de ejecución y el número total de picos generados por el modelo.


In [13]:
def evaluate_model(model, X_train, X_test, y_train, y_test):
    start_time = time.time()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    end_time = time.time()
    elapsed_time = end_time - start_time
    total_spikes = sum([sum(model.neuron.update(I=ft, dt=1) for ft in sample) for sample in X_test])
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy, elapsed_time, total_spikes

# Bloque principal.

- Nº de muestras: 10,000.

Código para generar datos, dividirlos en conjuntos de entrenamiento y prueba,
crear instancias de modelos, evaluar cada modelo y mostrar los resultados.

In [ ]:
num_samples = 10000

X, y = generate_data(num_samples)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

if_model = IFModel()
hh_model = HHModel()
nlif_model = NLIFModel()
fhn_model = FHNModel()
ifa_model = IFAModel()

if_accuracy, if_time, t1 = evaluate_model(if_model, X_train, X_test, y_train, y_test)
hh_accuracy, hh_time, t2 = evaluate_model(hh_model, X_train, X_test, y_train, y_test)
nlif_accuracy, nlif_time, t3 = evaluate_model(nlif_model, X_train, X_test, y_train, y_test)
fhn_accuracy, fhn_time, t4 = evaluate_model(fhn_model, X_train, X_test, y_train, y_test)
ifa_accuracy, ifa_time, t5 = evaluate_model(ifa_model, X_train, X_test, y_train, y_test)

print("Precisión del modelo IF:", if_accuracy)
print("Tiempo de ejecución del modelo IF:", if_time, "segundos")
print("Precisión del modelo HH:", hh_accuracy)
print("Tiempo de ejecución del modelo HH:", hh_time, "segundos")
print("Precisión del modelo NLIF:", nlif_accuracy)
print("Tiempo de ejecución del modelo NLIF:", nlif_time, "segundos")
print("Precisión del modelo FitzHugh-Nagumo:", fhn_accuracy)
print("Tiempo de ejecución del modelo FitzHugh-Nagumo:", fhn_time, "segundos")
print("Precisión del modelo Integrate-Fire con Adaptación:", ifa_accuracy)
print("Tiempo de ejecución del modelo Integrate-Fire con Adaptación:", ifa_time, "segundos")